In [1]:
from collections import namedtuple
from random import random
from random import shuffle

import numpy as np

In [2]:
Word = namedtuple('Word', ['written', 'phonetic'])


def load_words():
    """Loads the words from the CMU phonetic dictionary and parses them into Word objects
    
    Returns:
        list, of Word objects 
    """
    words = []
    bad_lines = 0
    with open('data/cmudict-0.7b', 'rb') as dict_handle:
        for line in dict_handle:
            try:
                line = line.decode("utf-8") 
            except:
                bad_lines += 1
                continue
                
            if line[0] == ';' or len(line) < 2:
                continue # Header or comment line

            line_chunks = str(line[:-1]).split(' ')
            written = line_chunks[0].lower()
            phonetic = [phone for phone in line_chunks[1:] if len(phone) >= 1]
            words.append(Word(written, phonetic))
            
    print('Number of lines that could not be parsed: ', bad_lines)
    return words

# Grab the words and check they loaded correctly.
words = load_words()
n_words = len(words)
sample_word = words[int(random() * len(words))]
print("""
      Found {n_words} words
      Sample word:
          {sample_word}
      """.format(**locals()))

Number of lines that could not be parsed:  1

      Found 133851 words
      Sample word:
          Word(written='bromont', phonetic=['B', 'R', 'OW1', 'M', 'AA2', 'N', 'T'])
      


In [12]:
class NameRecommender:
    """Base Class for producing name recommendations based on a list of liked
    words from a user. There are possible ways to upgrade this base class and
    they are marked with TODO.
    
    To use:
        # Initialize
        namer = NameRecommender(words, featurizer)
        
        # Example for pulling L2 recommendations for User 1
        user1_features = namer.construct_user_features(['fox', 'box'], ['hello', 'abracadabra'])
        namer.recommend_on_L2_similarity(user1_features, 10)
        
        # Example for pulling L-Infinity recommendations for User 1 after the above lines
        namer.recommend_on_Linf_similarity(user1_features, 10)
        
        # Example for pulling L2 recommendations for User 2
        user2_features = namer.construct_user_features(['hello', 'abracadabra'], ['fox', 'box'])
        namer.recommend_on_L2_similarity(user2_features, 10)
    """
    
    def __init__(self, words, featurizer):
        """Initialize the Recommender language and features
        
        Args:
            words: (list of Word objects), all words in this recommender's language
            featurizer: (function) takes a Word object and returns a np.array featurizing the word
        """
        self.words = words
        self.featurizer = featurizer
        self._construct_word_features()
        
        # Of the possible words limit to candidates to consider
        self.candidates = [
            word.written  
            for word in words
            if True # TODO remove undesireable words with non-alphabetic characters
        ] 
        
    def _construct_word_features(self):
        """Featurizes the dictionary for quick and easy comparison to later.
        """
        self.features = dict((word.written, self.featurizer(word))
                             for word in self.words)
        # TODO add normalization to self.featurizer
        
    def construct_user_features(self, liked_words, disliked_words):
        """Constructs the User Features based on liked and disliked words
        
        Args:
            liked_words: (list of str) the written words a user likes ie ['fox', 'box']
            disliked_words: (list of str) the written words a user dislikes ie ['hello', 'fellow']
            
        Returns:
            (np.array) average features of the user's words.
        """
        token_liked_features = [
            self.features[word]
            for word in liked_words
            if word in self.features
        ]
        liked_features = sum(token_liked_features) / float(len(token_liked_features))
        
        # TODO substract features based on disliked_words
        
        # TODO normalize the feature vector
        
        return liked_features
        
    def _recommend(self, similarity_function, user_features, n, threshold=0.0):
        """Core functionality that provides the recommendations based on the initialized
        recommender and the described user.
        
        Args:
            similarity_function: (function) takes in two feature vectors and returns a float
            user_features: (np.array) a feature vector representing the user's tastes
            n: (int), number of recommendations to return
            
        Kwargs:
            threshold: (float) the tolerance of the best score
            
        Returns:
            (list of str), a randomly ordered list of the candidates with the highest score
        """
        Rank = namedtuple('Rank', ['word', 'score'])
        
        # This is the costly step, comparing a user feature to all other features
        ranked_candidates = [
            Rank(candidate, similarity_function(self.features[candidate], user_features))
            for candidate in self.candidates
        ]
        ranked_candidates.sort(key=lambda candidate: candidate.score)
        
        # Find the best score and limit results to those within 
        best_score = ranked_candidates[0].score
        top_candidates = [
            candidate
            for candidate in ranked_candidates
            if candidate.score <= best_score + threshold
        ]
        print('Found ', len(top_candidates), 'top candidates to use.')
        
        # Shuffle the results for flavor and return TODO enable to change up results
        #shuffle(top_candidates)
        return [candidate.word for candidate in top_candidates[:n]]
        
    def recommend_on_L2_similarity(self, user_features, n, threshold=0.0):
        """Users the L2 norm for detecting the similarity between two feature vectors and
        finds the best recommendations accordingly.
        
        Args:
            user_features: (np.array) a feature vector representing the user's tastes
            n: (int), number of recommendations to return
            
        Kwargs:
            threshold: (float), the tolerance with which return results scoring within the
                threhold of the best score
            
        Returns:
            (list of str), a randomly ordered list of the candidates with the highest score
        """
        def L2(features_1, features_2):
            return sum((features_1 - features_2)**2)**0.5
        
        return self._recommend(L2, user_features, n, threshold)
                
    def recommend_on_Linf_similarity(self, user_features, n, threshold=0.0):
        """Users a modification of the L-inf norm for detecting the similarity between
        two feature vectors and finds the best recommendations accordingly.
        
        Args:
            user_features: (np.array) a feature vector representing the user's tastes
            n: (int), number of recommendations to return
            
        Kwargs:
            threshold: (float), the tolerance with which return results scoring within the
                threhold of the best score

        Returns:
            (list of str), a randomly ordered list of the candidates with the highest score
        """
        def Linf(features_1, features_2):
            return sum(-1
                       for (f1, f2) in zip(features_1, features_2)
                       if f1 > 0 and f2 > 0
                      )
        
        return self._recommend(Linf, user_features, n, threshold)
    
    def recommend_on_TODO_similarity(self, user_features, n):
        """If you'd like try a custom similarity ranker. See the above L2 and Linf
        rankers for the implementation pattern.
        """
        raise Exception('Not Implemented')


"""
Handy Dandy local variable for creating featurizers.
Create a mapping from phones to a feature number.
"""
def get_all_phones(words):
    """Gets
    """
    phones = set([])
    for word in words:
        phones.update(word.phonetic)
        
    return list(phones)

all_phones = get_all_phones(words)
phone_to_id = dict((phone, i) for (phone, i) in zip(all_phones, range(len(all_phones))))

In [13]:
def rhyming_featurizer(word):
    """Basic featurizer looking for words that end the same.
    
    Args:
        word: (Word object) 
        
    Returns:
        (np.array) with non-zero entries based on the ending phones of the word and its length
    """
    features = np.zeros(len(phone_to_id) + 1)
    
    # TODO use the last 2 phones for more exact rhymes
    features[phone_to_id[word.phonetic[-1]]] = 1
    
    # Adding an extra feature to keep words to roughly the same length
    features[-1] = len(word.phonetic)
    
    return features

rhyming_namer = NameRecommender(words, rhyming_featurizer)

In [14]:
"""
Test out the Rhyming Namer!
"""
liked_words = [
    'fox',
    'box',
    'tax',
    'fax',
    'sacks'
]
disliked_words = [
    'hello',
    'fellow',
]
fox_features = rhyming_namer.construct_user_features(liked_words, disliked_words)
fox_L2_rhymes = rhyming_namer.recommend_on_L2_similarity(fox_features, 10)
fox_Linf_rhymes = rhyming_namer.recommend_on_Linf_similarity(fox_features, 10)
print("""
Rhyming Recommendations with only the highest scores based on seed words: {liked_words}.

Based on L2 Similarity:
    {fox_L2_rhymes}
    
Based on Linf Similarity:
    {fox_Linf_rhymes}
""".format(**locals()))

Found  1307 top candidates to use.
Found  11319 top candidates to use.

Rhyming Recommendations with only the highest scores based on seed words: ['fox', 'box', 'tax', 'fax', 'sacks'].

Based on L2 Similarity:
    ["'course", '+plus', 'abbas', 'abts', 'abyss', "act's", 'acts', 'addis', 'aegis', 'aerts']
    
Based on Linf Similarity:
    ['"in-quotes', "'course", "'s", '+plus', '...ellipsis', 'aardvarks', 'aase', 'abacus', 'abandonments', 'abatements']



In [15]:
def matching_featurizer(word):
    """Featurizer trying to match as many sounds as possible.
    
    Args:
        word: (Word object) 
        
    Returns:
        (np.array) with non-zero entries based on the ending phones of the word and its length
    """
    features = np.zeros(len(phone_to_id) + 1)
    for phone in word.phonetic:
        features[phone_to_id[phone]] += 1

    return features

matching_namer = NameRecommender(words, matching_featurizer)

In [18]:
# Try a matching featurizer to get as many sounds as possible in the recommendations
liked_words = [
    'glass',
]
disliked_words = [
    'hello',
    'there',
]
glass_features = matching_namer.construct_user_features(liked_words, disliked_words)
glass_L2_matches = matching_namer.recommend_on_L2_similarity(glass_features, 10)
print("""
Matching Recommendations based on seed words: {liked_words}.

Top Matches Based on L2 Similarity:
    {glass_L2_matches}
""".format(**locals()))

Found  3 top candidates to use.

Matching Recommendations based on seed words: ['glass'].

Top Matches Based on L2 Similarity:
    ['glas', 'glass', 'slag']



In [20]:
# Let's relax the threshold

glass_features = matching_namer.construct_user_features(liked_words, disliked_words)
glass_L2_matches = matching_namer.recommend_on_L2_similarity(glass_features, 10, 1.0)
print("""
Matching Recommendations based on seed words: {liked_words}.

Top Matches Based on L2 Similarity:
    {glass_L2_matches}
""".format(**locals()))

Found  19 top candidates to use.

Matching Recommendations based on seed words: ['glass'].

Top Matches Based on L2 Similarity:
    ['glas', 'glass', 'slag', 'gal', 'gallus', 'galus', 'gas', 'gass', 'glance', 'glassed']

